In [1]:
import pandas as pd


In [2]:
df = pd.read_csv(r"/content/fake & true.csv")
df.head()

,Unnamed: 0,title,text,subject,date,labels
0,0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [3]:
df = df.sample(frac=1,random_state=42).reset_index(drop=True)
df.head(10)

,Unnamed: 0,title,text,subject,date,labels
0,22216,BREAKING: GOP Chairman Grassley Has Had Enoug...,"Donald Trump s White House is in chaos, and th...",News,"July 21, 2017",0
1,27917,Failed GOP Candidates Remembered In Hilarious...,Now that Donald Trump is the presumptive GOP n...,News,"May 7, 2016",0
2,25007,Mike Pence’s New DC Neighbors Are HILARIOUSLY...,Mike Pence is a huge homophobe. He supports ex...,News,"December 3, 2016",0
3,1377,California AG pledges to defend birth control ...,SAN FRANCISCO (Reuters) - California Attorney ...,politicsNews,"October 6, 2017",1
4,32476,AZ RANCHERS Living On US-Mexico Border Destroy...,Twisted reasoning is all that comes from Pelos...,politics,"Apr 25, 2017",0
5,5766,"As private lawyer, Trump high court pick was f...",WASHINGTON (Reuters) - As a lawyer in private ...,politicsNews,"February 1, 2017",1
6,16230,Yemeni Salafist imam killed in Aden: sources,ADEN (Reuters) - A Salafist imam was shot dead...,worldnews,"October 28, 2017",1
7,20911,FBI says witnesses in U.S. probe into Malaysia...,KUALA LUMPUR (Reuters) - Potential witnesses t...,worldnews,"September 6, 2017",1
8,35838,An Easy To Read Chart Shows How Bernie Sanders...,The goal of socialism is communism. -Vladimi...,politics,"Feb 24, 2016",0
9,33155,MMA FIGHTER JAKE SHIELDS Embarrasses Cowards I...,Opposing views and beliefs has much of this co...,politics,"Feb 4, 2017",0


In [4]:
X = df.drop('labels',axis=1)
y = df['labels']

In [5]:
y.value_counts()

,count
labels,
0,23481
1,21417


In [40]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Dense,Bidirectional,Embedding,Dropout

In [7]:
!pip install gensim

In [8]:
from gensim.models import Word2Vec

In [9]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
messages = X.copy()

In [11]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
corpus = []

for i in range(0,len(messages)):
  review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]

  corpus.append(review)

In [12]:
corpus[0]

['break',
 'gop',
 'chairman',
 'grassley',
 'enough',
 'demand',
 'trump',
 'jr',
 'testimoni']

In [13]:
model = Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=2, workers=4)
word_vectors = model.wv

In [14]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')


[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [15]:
import numpy as np

def document_vector(doc):
    doc = [word for word in doc if word in wv]  # keep only known words
    if len(doc) == 0:
        return np.zeros(300)  # if no known words
    return np.mean(wv[doc], axis=0)

In [16]:
vec = document_vector(corpus[0])


In [19]:
print(vec.shape)

(300,)


In [20]:
X.head()

,Unnamed: 0,title,text,subject,date
0,22216,BREAKING: GOP Chairman Grassley Has Had Enoug...,"Donald Trump s White House is in chaos, and th...",News,"July 21, 2017"
1,27917,Failed GOP Candidates Remembered In Hilarious...,Now that Donald Trump is the presumptive GOP n...,News,"May 7, 2016"
2,25007,Mike Pence’s New DC Neighbors Are HILARIOUSLY...,Mike Pence is a huge homophobe. He supports ex...,News,"December 3, 2016"
3,1377,California AG pledges to defend birth control ...,SAN FRANCISCO (Reuters) - California Attorney ...,politicsNews,"October 6, 2017"
4,32476,AZ RANCHERS Living On US-Mexico Border Destroy...,Twisted reasoning is all that comes from Pelos...,politics,"Apr 25, 2017"


In [23]:
X_raw = np.array([document_vector(doc) for doc in corpus])

In [27]:
corpus[0]

['break',
 'gop',
 'chairman',
 'grassley',
 'enough',
 'demand',
 'trump',
 'jr',
 'testimoni']

In [28]:
X_raw = X['title']

In [29]:
X_raw.head()

,title
0,BREAKING: GOP Chairman Grassley Has Had Enoug...
1,Failed GOP Candidates Remembered In Hilarious...
2,Mike Pence’s New DC Neighbors Are HILARIOUSLY...
3,California AG pledges to defend birth control ...
4,AZ RANCHERS Living On US-Mexico Border Destroy...


In [30]:
X = np.array([document_vector(doc) for doc in corpus])

In [32]:
X.shape

(44898, 300)

In [33]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [38]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_words = 20000
max_len = 200

tokenizer = Tokenizer(num_words=max_words,oov_token="OOV")
tokenizer.fit_on_texts(X_raw)

sequences = tokenizer.texts_to_sequences(X_raw)
X = pad_sequences(sequences,maxlen=max_len,padding='post')

In [39]:
import numpy as np

embedding_dim = 300
word_index = tokenizer.word_index
embedding_matrix = np.zeros((max_words, embedding_dim))

for word, i in word_index.items():
    if i < max_words and word in wv:
        embedding_matrix[i] = wv[word]

In [43]:
my_model = Sequential()
my_model.add(Embedding(input_dim=max_words, output_dim=embedding_dim,weights=[embedding_matrix], trainable=False))
my_model.add(Bidirectional(LSTM(128,return_sequences=True)))
my_model.add(Dropout(0.5))
my_model.add(Bidirectional(LSTM(64)))
my_model.add(Dropout(0.5))
my_model.add(Dense(1,activation='sigmoid'))

In [44]:
my_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
history = my_model.fit(X,y,epochs=5,batch_size=64,validation_split=0.2)

Epoch 1/5
562/562 ━━━━━━━━━━━━━━━━━━━━ 35s 47ms/step - accuracy: 0.8854 - loss: 0.2955 - val_accuracy: 0.9356 - val_loss: 0.1644
Epoch 2/5
562/562 ━━━━━━━━━━━━━━━━━━━━ 38s 47ms/step - accuracy: 0.9390 - loss: 0.1567 - val_accuracy: 0.9500 - val_loss: 0.1331
Epoch 3/5
562/562 ━━━━━━━━━━━━━━━━━━━━ 26s 47ms/step - accuracy: 0.9557 - loss: 0.1161 - val_accuracy: 0.9558 - val_loss: 0.1139
Epoch 4/5
562/562 ━━━━━━━━━━━━━━━━━━━━ 42s 49ms/step - accuracy: 0.9626 - loss: 0.0995 - val_accuracy: 0.9537 - val_loss: 0.1185
Epoch 5/5
562/562 ━━━━━━━━━━━━━━━━━━━━ 40s 48ms/step - accuracy: 0.9692 - loss: 0.0839 - val_accuracy: 0.9586 - val_loss: 0.1013


In [45]:
loss,acc = my_model.evaluate(X,y)
print(f"Accuracy: {acc:.2f}")

1404/1404 ━━━━━━━━━━━━━━━━━━━━ 21s 15ms/step - accuracy: 0.9757 - loss: 0.0694
Accuracy: 0.97


In [48]:
def predict_headline(text):
    seq = tokenizer.texts_to_sequences([text])
    pad = pad_sequences(seq, maxlen=max_len, padding='post')
    prob = my_model.predict(pad)[0][0]
    label = "Real" if prob > 0.5 else "Fake"
    return label, prob

label, prob = predict_headline("Why Pak Dropped Bombs From Chinese Jets, Killing 30 In Khyber Pakhtunkhwa")
print(label, prob)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Real 0.85904485
